# Algo 2 – Python

## Cours 1 – rappels

###  Master Humanités Numériques du CESR

# Regex et parseurs

Aujourd’hui deux notions/outils qui pourront vous être utiles : les expressions régulières (regex) et les parseurs. On aura sans doute pas le temps de tout voir en détail, vous pourrez poursuivre par vous-même.

## Regex

Le langage formel des expressions régulières (aka regex, *regular expression* en anglais) est un outil puissant et expressif pour la recherche de chaînes de caractères. Il permet de définir un motif (*pattern*) de recherche qui porte sur une chaîne de caractères.

Exemples : 
 - "tubes?" -> chaîne de caractère "tube" ou "tubes"
 - ".+eur\b" -> chaîne se terminant par "eur"
 - "\d{4}-\d{2}-\d{2}" -> date au format ISO 8601 : 2024-03-27 (4 entiers, un tiret, 2 entiers, un tiret, 2 entiers)
 - …

Les expressions régulières sont supportées par la plupart de langages de programmation. Vous les retrouvez dans les éditeurs de texte, dans les traitements de textes et dans pas mal de moteurs de recherche.  
Comme c’est très utilisé il existe beaucoup beaucoup de sites bien faits :

- [https://regex101.com/](https://regex101.com/) est un site très bien fait pour se former et tester des regex. Allons y tester les exemples vus plus haut.
- Il y a de très bons supports de documentation sur les regex, autant en profiter. Par exemple [https://quickref.me/regex.html](https://quickref.me/regex.html)  
- Vous pouvez aussi vous auto former avec [https://regexone.com/](https://regexone.com/) ou le magnifique [https://regexcrossword.com](https://regexcrossword.com)

En python ça se passe avec le module `re` (livré avec la distribution standard) ou `regex` (module externe à installer). On va s’en tenir à `re` ici.

- `re` est un module important, vous devez en lire la [documentation](https://docs.python.org/3/library/re.html), absolument
- La doc officielle est parfois aride, ce [howto](https://docs.python.org/3/howto/regex.html) rédigé par A.M. Kuchling est plus digeste

A minima vous devez connaître les fonctions :

- `findall` : trouve toutes les occurences du motif, retourne une liste de chaînes trouvées
- `search` : trouve le motif, retourne un objet Match, None sinon
- `match` : détermine si le motif est présent au début de la chaîne, retourne un objet Match, None sinon
- `split` : découpe une chaîne selon un motif, retourne une liste de chaînes
- `sub` : remplace les occurences d'un motif par une chaîne de remplacement
- `compile` : compilation d'un motif (pattern), retourne un objet Pattern

In [5]:
## Exemple avec "chaîne se terminant en 'eur'"

import re

first_str = "Avec toutes ces histoires j’ai peur de la clameur de la foule."
second_str = "Avec toutes ces histoires j’ai envie d’une tasse de café."
if re.search(r".+eur\b", first_str):
    print("Trouvé")
else:
    print("Pas trouvé")

Trouvé


### Objet `Match`
`re.search` et `re.match` renvoient un objet de type `re.Match`

In [6]:
m = re.match(r'\w', 'spam')
type(m)

re.Match

Si le motif est trouvé, l'objet est évalué comme vrai (*truthy*) s'il est testé mais il contient plus d'informations :

- `m.group()` la chaîne trouvée (matchée)
- `m.start()` l'indice de la position initiale de la chaîne
- `m.end()` l'indice de la position finale de la chaîne
- `m.span()` le tuple indice début, fin de la chaîne

In [16]:
m = re.search(r"l[ae]s?", "Après la pluie, le beau temps")
m.group()

'la'

Si le motif comporte des groupes de capture :
- `m.group(1)` renvoie la chaîne correspond au 1er groupe, etc.
- `m.groups()` renvoie un tuple comportant autant d'éléments qu'il y a de groupes

In [8]:
m = re.search(r"(l[ae]s?)\s(\w+)", "Après la pluie, le beau temps")
m.groups()

('la', 'pluie')

### ✍️  Exo ✍️

- Trouver les noms de personnes ou les noms de lieux dans ces paragraphes :
  
  « Famed American artist and sculptor Richard Serra, known for turning curving walls of rusting steel and other malleable materials into large-scale pieces of outdoor artwork that are now dotted across the world, died Tuesday at his home in Long Island, New York. He was 85.

    Considered one of his generation's most preeminent sculptors, the San Francisco native originally studied painting at Yale University but turned to sculpting in the 1960s, inspired by trips to Europe.

    His death was confirmed Tuesday night by his lawyer, John Silberman, whose firm is based in New York. He said the cause of death was pneumonia. »  
  (extrait de https://www.npr.org/2024/03/26/1241104634/richard-serra-dead)
- Trouver les liens hypertextes dans la page `https://www.reddit.com/r/Python/` et les afficher sous la forme `ancre: url`

## Parseurs

Dans ce notebook nous utiliserons le parseur [lxml](http://lxml.de/) qui est un binding de libxml2 et [Beautiful Soup](https://www.crummy.com/software/BeautifulSoup/) 

Cette partie date de décembre 2020. Je me suis permis de la reprendre telle quelle. L’exemple avec les chansons de NPR fonctionne toujours. Nous allons tout de même essayer de le mettre à jour avec les chansons de 2023.

## Parser de l'html

Beautiful Soup nous permet de parser simplement du contenu html. Même si le contenu est mal formé, le module bs reconstitue un arbre et offre des fonctions faciles à utiliser pour parcourir l'arbre ou y rechercher des éléments.  
Beautiful Soup n'est pas un parseur, il utilise les parseurs et offre une API simplifiée à ses utilisateurs.

Nous travaillerons directement avec du contenu en ligne à l'aide du module `requests`. Si vous ne l'avez pas en magasin, installez le.  
Décembre c'est le mois des listes, nous nous attacherons à la liste des 100 meilleures chansons de l'année de NPR la radio publique américaine : https://www.npr.org/2020/12/03/931771524/the-100-best-songs-of-2020-page-1  
Allez y faire un tour.

In [28]:
import requests
from bs4 import BeautifulSoup

url = "https://www.npr.org/2020/12/03/931771524/the-100-best-songs-of-2020-page-1"
r = requests.get(url)
soup = BeautifulSoup(r.text, 'lxml')

Voilà nous avons maintenant un objet `soup` de classe Beautiful Soup.  
La [doc](https://www.crummy.com/software/BeautifulSoup/bs4/doc/) est très claire.

In [29]:
url = "https://www.npr.org/2020/12/03/931771524/the-100-best-songs-of-2020-page-1"
r = requests.get(url, cookies=cookies)
soup = BeautifulSoup(r.text, 'lxml')
# je cherche l'élement avec le tag 'title'
print(soup.title)
# le tag de l'élément
print(soup.title.name)
# le contenu textuel de l'élément
print(soup.title.string)

<title>NPR's 100 Best Songs Of 2020, Ranked : NPR</title>
title
NPR's 100 Best Songs Of 2020, Ranked : NPR


On cherche à récupérer la liste des 100 chansons : rang, titre, interprète. Puis on les affichera par ordre croissant.  
Il faut inspecter le code source et repérer les élements html et les classes utilisées pour le contentu qui nous intéresse.  
Exemple avec le premier, enfin le 100ème : BTS. Dynamite. 🎶 Cos I… I… I'm in the stars tonight 🎸 🎶

```html
<h6 class="edTag"><a id="bts" class="anchor"> </a>100.</h6>
<h3 class="edTag">BTS</h3>
<h3 class="edTag">"Dynamite"</h3>
```

In [25]:
# Exemple pour récupérer les éléments h6 class='edTag'
for item in soup.find_all('h6', attrs={'class':'edTag'}):
    print(item.text)
# on peut aussi utiliser la notation suivante
#for item in soup.find_all('h6', class_="edTag"):
#    print(item.text)

### ✍️  Exo ✍️
Maintenant à vous de jouer. Il faut parcourir la [doc](https://www.crummy.com/software/BeautifulSoup/bs4/doc/) pour trouver la fonction qui vous permettra de récupérer les deux éléments h3 suivants et seulement ceux-là. 
Je vous laisse chercher un peu ![Alt Text](https://media.giphy.com/media/l2SpZkQ0XT1XtKus0/giphy.gif)

Encore un peu ![on cherche](https://media.giphy.com/media/JO9pi3EeHzyBu5YNMK/giphy.gif)

In [6]:
for item in soup.find_all('h6', attrs={'class':'edTag'}):
    rank = item.text
    h3s = item.find_next_siblings('h3', attrs={'class':'edTag'}, limit=2)
    artist = h3s[0].text
    title = h3s[1].text
    print(f"{rang}, {artist}, {title}")

 100., BTS, "Dynamite"
 99., Sturgill Simpson, "Living The Dream"
 98., Ariana Grande, "pov"
 97., Busta Rhymes (feat. Kendrick Lamar), "Look Over Your Shoulder"
 96. , Chicano Batman , "Color my life"
 95., Tiwa Savage , "Dangerous Love (DJ Tunez & D3an Remix)"
 94. , Breland (feat. Sam Hunt), "My Truck (Remix)"
 93., Leon Bridges (feat. Terrace Martin), "Sweeter"
 92., Yumi Zouma, "Cool For A Second"
 91., Hayley Williams, "Simmer"
 90., The 1975, "If You're Too Shy (Let Me Know)"
 89., Swamp Dogg, "Billy"
 88., Roomful Of Teeth, "Just Constellations No. 1, The Opening Constellation (Summer)"
 87., GIVĒON, "Still Your Best"
 86., Moneybagg Yo, "Said Sum"
 85., Rita Indiana (feat. Kiko El Crazy), "Mandinga Times"
 84., Ashnikko (feat. Grimes), "Cry"
 83., Ultraísta, "Tin King"
 82., Sweeping Promises, "Hunger For A Way Out"
 81., Arlo McKinley, "Die Midwestern"


### ✍️  Exo ✍️

C'est bien mais pas suffisant. Il reste :
  1. nettoyer les rangs, c-a-d supprimmer le point qui traîne à la fin et l'espace des fois.
  2. stocker les infos dans une donnée structurée. Utilisez une classe à vous ou plus simple un `namedtuple`
  3. faire l'opération pour toutes les pages web afin d'avoir le classement de 1 à 100.

In [7]:
urls = [
    "https://www.npr.org/2020/12/03/931771524/the-100-best-songs-of-2020-page-1",
    "https://www.npr.org/2020/12/03/934634561/the-100-best-songs-of-2020-page-2?utm_source=page1&utm_campaign=next&utm_term=bottom&utm_medium=internal",
    "https://www.npr.org/2020/12/03/934634607/the-100-best-songs-of-2020-page-3?utm_source=page2&utm_campaign=next&utm_term=bottom&utm_medium=internal",
    "https://www.npr.org/2020/12/03/934634855/the-100-best-songs-of-2020-page-4?utm_source=page3&utm_campaign=next&utm_term=bottom&utm_medium=internal",
    "https://www.npr.org/2020/12/03/934634998/the-100-best-songs-of-2020-page-5?utm_source=page4&utm_campaign=next&utm_term=bottom&utm_medium=internal"
]
# à vous

In [ ]:
from collections import namedtuple

class Song():
    """
    A song with 3 infos : artist, title, rank (according to NPR list)
    """
    def __init__(self, artist, title, rank):
        self.artist = artist
        self.title = title
        self.rank = rank

songs = []
for url in urls:
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'lxml')
    for item in soup.find_all('h6', attrs={'class':'edTag'}):
        rank = item.text.rstrip(". ").lstrip()
        h3s = item.find_next_siblings('h3', attrs={'class':'edTag'}, limit=2)
        artist = h3s[0].text
        title = h3s[1].text
        songs.append(Song(artist, title, rank))

for song in sorted(songs, key=lambda x:int(x.rank)):
    print(song.rank, song.artist, song.title)

### ✍️  Exo ✍️

Reprendre et adapter le script ci-dessus pour la page : [https://www.npr.org/2023/12/12/1215355752/best-songs-2023](https://www.npr.org/2023/12/12/1215355752/best-songs-2023)

## Parser de l'xml

Nous allons travailler sur un fichier au format TEI extrait du corpus *Corpus 14*  
PRAXILING - UMR 5267 (PRAXILING) (2014). Corpus 14 [Corpus]. ORTOLANG (Open Resources and TOols for LANGuage) - www.ortolang.fr, https://hdl.handle.net/11403/corpus14/v1.  

Le fichier se nomme ``josephine-1-150119.xml``. Il s'agit d'une lettre d'une femme de soldat à son époux.  
Nous allons extraire du fichier TEI les informations suivantes :  
- titre (``/TEI/teiHeader/fileDesc/titleStmt/title``)
- source (``/TEI/teiHeader/fileDesc/sourceDesc/p``)
- contenu de la lettre (``/TEI/text/body``)

### Avec lxml

Pourquoi `lxml` et pas `xml.etree.ElementTree` ? Parce que : [1](http://lxml.de/intro.html) et surtout [2](http://lxml.de/performance.html)  
La bonne nouvelle c'est que votre code sera aussi compatible avec `xml.etree.ElementTree` ou `xml.etree.cElementTree` parce que xml utilise l'API ElementTree. Sauf pour la méthode `xpath` qui est propre à `libxml`.

In [9]:
from lxml import etree
tree = etree.parse('data/josephine-1-150119.xml')
root = tree.getroot()

# Parcours des enfants de la racine (commentaires et éléments)
for child in root:
    print(child.tag)

<cyfunction Comment at 0x7f7856fb3030>
{http://www.tei-c.org/ns/1.0}teiHeader
<cyfunction Comment at 0x7f7856fb3030>
{http://www.tei-c.org/ns/1.0}facsimile
{http://www.tei-c.org/ns/1.0}text


Le fichier utilise l'espace de nom TEI : ``<TEI xmlns="http://www.tei-c.org/ns/1.0">``, nous devrons l'indiquer dans nos instructions de recherche.  
Voyons ça pour le titre (``/TEI/teiHeader/fileDesc/titleStmt/title``)

In [10]:
# la méthode find renvoie le premier élément qui correspond au chemin argument (ElementPath et non Xpath)
title = root.find("./tei:teiHeader/tei:fileDesc/tei:titleStmt/tei:title", namespaces={'tei':"http://www.tei-c.org/ns/1.0"})
print("Tag : {}".format(title.tag))
print("Texte : {}".format(title.text))

Tag : {http://www.tei-c.org/ns/1.0}title
Texte : Joséphine Pouchet à son époux le 19-01-1915 depuis Baillargues


Même traitement pour la source :

In [11]:
source = root.find("./tei:teiHeader/tei:fileDesc/tei:sourceDesc/tei:p", namespaces={'tei':"http://www.tei-c.org/ns/1.0"})
print("Tag : {}".format(source.tag))
print("Texte : {}".format(source.text))

Tag : {http://www.tei-c.org/ns/1.0}p
Texte : Correspondance de Joséphine Pouchet, numérisée par les Archives Départementales de l'Hérault.


lxml a aussi une méthode ``xpath`` qui permet d'utiliser directement des expressions xpath (sans oublier les espace de noms pour notre fichier) :

In [12]:
source = root.xpath("/tei:TEI/tei:teiHeader/tei:fileDesc/tei:sourceDesc/tei:p", namespaces={'tei':'http://www.tei-c.org/ns/1.0'})
print(type(source)) #xpath retourne une liste
print(source[0].text)
#ou bien
source = root.xpath("/tei:TEI/tei:teiHeader/tei:fileDesc/tei:sourceDesc/tei:p/text()", namespaces={'tei':'http://www.tei-c.org/ns/1.0'})
print(source[0])

<class 'list'>
Correspondance de Joséphine Pouchet, numérisée par les Archives Départementales de l'Hérault.
Correspondance de Joséphine Pouchet, numérisée par les Archives Départementales de l'Hérault.


Pour le contenu il faut ruser. La difficulté ici tient à l'utilisation d'élements `<lb/>` de type 'milestones' pour noter les retours à la ligne :  
```xml
<p>
je reponse a ton aimableux lettres<lb/>
que nous a fait plaisir en naprenas<lb/>
que tu et enbonne santes car il<lb/>
anais de maime pour nous<lb/>
</p>
```

In [13]:
# la méthode findall renvoie une liste avec tous les éléments correspondant au chemin argument
body = root.findall("./tei:text/tei:body/tei:p", namespaces={'tei':"http://www.tei-c.org/ns/1.0"})
for elem in body:
    print(elem.text)


cher Laurent

je reponse a ton aimableux lettres

cher Laurent je repons a la cartes


Ici on ne récupère que les noeuds text précédant les éléments `<lb/>`  
Une requête `xpath` va nous permettre de récupérer tous les noeuds text

In [14]:
body = root.xpath("//tei:text/tei:body//text()", namespaces={'tei':"http://www.tei-c.org/ns/1.0"})
for text in body:
    print(text, end="")



Baillargues Le 19 janvier 1915


cher Laurent


je reponse a ton aimableux lettres
que nous a fait plaisir en naprenas
que tu et enbonne santes car il
anais de maime pour nous


cher Laurent je repons a la cartes
de ma mère quelles et venue au
jourdhui pour de faire partire
partir un paquet quil aillae
les chosette sausice chocolas une
paire de chosette pour Louis
je pense que vous magerè ensenbleus
tu feras repons a la maison te
suite que tu rese vras le paquet
je te dirais que ten le midi il
fait frois il fait du vent glasais
et toi au pas de calais tu nous
dit quil pleus mai tu nous parles
pas si tu a ases pour te garendir
du froit cil te maque quelles chose

tu nas que ledire quon de len verras
verras tu nous dit que charles ta
Ecrie et ta soeux et ta dit que
je lui et envoiez la photot
plurien a te dire pour le moment
que de ten voiez une grose
carriese de tous et boutounase
de ton petit enge 
adorè Albert encorre une foi
te plui Milles baisées te tous
ta fenme pour la vie
Josep

## avec DOM

L'API `ElementTree` est propre à Python, `DOM` est une API indépendante d'un langage de programmation. Il existe des implémentations `DOM` dans la plupart des langages de programmation modernes.  

In [15]:
from xml.dom import minidom
dom = minidom.parse("data/josephine-1-150119.xml")
# l'objet Document
dom

In [16]:
title = dom.getElementsByTagNameNS("http://www.tei-c.org/ns/1.0", 'title')[0] # un seul élément 'title' dans le document
print(title) # title est un objet Element, pour accèder au contenu textuel il faut récupérer le noeud texte
print(title.lastChild.nodeName)
print(title.lastChild.nodeValue)

<DOM Element: title at 0x7f785421d2d0>
#text
Joséphine Pouchet à son époux le 19-01-1915 depuis Baillargues


idem pour la source, sauf qu'on ne peut pas se permettre de rechercher tous les éléments `p`.  
Il faut trouver l'élément `p` fils de `sourceDesc`

In [17]:
sourceDesc = dom.getElementsByTagNameNS("http://www.tei-c.org/ns/1.0", 'sourceDesc')[0]
for node in sourceDesc.childNodes:
    if node.localName == "p":
        print(node.lastChild.nodeValue)

Correspondance de Joséphine Pouchet, numérisée par les Archives Départementales de l'Hérault.


Et maintenant le contenu et ses éléments milestones.  
Pour garder la forme vous réécrirez les boucles `for` suivies de `if` en listes en compréhension.

In [19]:
body = dom.getElementsByTagNameNS("http://www.tei-c.org/ns/1.0", 'body')[0]
for node in body.childNodes:
    if node.localName == "p" or "opener":
        for in_node in node.childNodes:
            if in_node.nodeName == "#text":
                print(in_node.nodeValue, end="")


Baillargues Le 19 janvier 1915

cher Laurent

je reponse a ton aimableux lettres
que nous a fait plaisir en naprenas
que tu et enbonne santes car il
anais de maime pour nous

cher Laurent je repons a la cartes
de ma mère quelles et venue au
jourdhui pour de faire 
partir un paquet quil 
les chosette sausice chocolas une
paire de chosette pour Louis
je pense que vous magerè ensenbleus
tu feras repons a la maison te
suite que tu rese vras le paquet
je te dirais que ten le midi il
fait frois il fait du vent glasais
et toi au pas de calais tu nous
dit quil pleus mai tu nous parles
pas si tu a ases pour te garendir
du froit cil te maque quelles chose

tu nas que ledire quon de len 
verras tu nous dit que charles ta
Ecrie et ta soeux et ta dit que
je lui et envoiez la photot
plurien a te dire pour le moment
que de ten voiez une grose
carriese de tous et boutounase
de ton petit enge 
adorè Albert encorre une foi
te plui Milles baisées te tous
ta fenme pour la vie
Josephine Pouchet
bien le bo

## Avec lxml et Beautiful Soup

In [20]:
from bs4 import BeautifulSoup

with open("data/josephine-1-150119.xml") as fp:
    soup = BeautifulSoup(fp, 'lxml')

In [21]:
soup.title.text

'Joséphine Pouchet à son époux le 19-01-1915 depuis Baillargues'

In [22]:
soup.sourcedesc.p.text

"Correspondance de Joséphine Pouchet, numérisée par les Archives Départementales de l'Hérault."

Pour le contenu de la lettre il y a la merveilleuse fonction `get_text()`

In [23]:
soup.get_text?

Signature:
soup.get_text(
    separator='',
    strip=False,
    types=<object object at 0x7f787458c700>,
)
Docstring:
Get all child strings of this PageElement, concatenated using the
given separator.

:param separator: Strings will be concatenated using this separator.

:param strip: If True, strings will be stripped before being
    concatenated.

:param types: A tuple of NavigableString subclasses. Any
    strings of a subclass not found in this list will be
    ignored. Although there are exceptions, the default
    behavior in most cases is to consider only NavigableString
    and CData objects. That means no comments, processing
    instructions, etc.

:return: A string.
File:      /usr/lib/python3/dist-packages/bs4/element.py
Type:      method

In [24]:
text = soup.find('text')
print(text.getText())





Baillargues Le 19 janvier 1915


cher Laurent


je reponse a ton aimableux lettres
que nous a fait plaisir en naprenas
que tu et enbonne santes car il
anais de maime pour nous


cher Laurent je repons a la cartes
de ma mère quelles et venue au
jourdhui pour de faire partire
partir un paquet quil aillae
les chosette sausice chocolas une
paire de chosette pour Louis
je pense que vous magerè ensenbleus
tu feras repons a la maison te
suite que tu rese vras le paquet
je te dirais que ten le midi il
fait frois il fait du vent glasais
et toi au pas de calais tu nous
dit quil pleus mai tu nous parles
pas si tu a ases pour te garendir
du froit cil te maque quelles chose

tu nas que ledire quon de len verras
verras tu nous dit que charles ta
Ecrie et ta soeux et ta dit que
je lui et envoiez la photot
plurien a te dire pour le moment
que de ten voiez une grose
carriese de tous et boutounase
de ton petit enge 
adorè Albert encorre une foi
te plui Milles baisées te tous
ta fenme pour la vie
Jos

lxml est rapide, Beautiful Soup simple à utiliser. Le combo diablement efficace.

Il y a un autre module super pour le web que nous ne verrons pas ici mais que je me dois de vous indiquer : https://selenium-python.readthedocs.io/  
Selenium va vous permettre d'automatiser des actions sur un navigateur. Je vous conseille d'essayer, c'est assez plaisant de voir votre navigateur piloté par un script.